In [31]:
import pprint
import numpy as np
import riskslim
import pickle
import pandas as pd
from riskslim.utils import *
from sklearn.metrics import roc_auc_score, confusion_matrix

In [32]:
# Load train dataset
df = pd.read_csv('data_train_infection.csv')
# df = pd.read_csv('data_train_rebleeding.csv')
# df = pd.read_csv('data_train_mortality.csv')

# Define target column
target_col = "infection"
# target_col = "rebleeding"
# target_col = "mortality"

# Identify the feature columns
feature_cols = [col for col in df.columns if col != target_col]

# Add an intercept column (all ones)
intercept_col = "(Intercept)"
df[intercept_col] = 1.0

# Prepend the intercept to the feature list
variable_names = [intercept_col] + feature_cols

# Create the feature matrix X and target vector Y
X = df[variable_names].values
y = df[target_col].values

# Create the data dictionary as expected by RiskSLIM
data = {
    'variable_names': variable_names,
    'X': X,
    'Y': y,
    'outcome_name': target_col 
}

data['Y'] = np.array(data['Y']).reshape(-1, 1)

print("Variable names:", data['variable_names'])
print("X shape:", data['X'].shape)
print("Y shape:", data['Y'].shape)


Variable names: ['(Intercept)', 'Antibiotic_prophylaxis', 'Sex', 'HCC', 'Ascites', 'Hepatic_encephalopathy', 'Prior_SBP', 'ICU_admission', 'Age_bin_0', 'Age_bin_1', 'Blood_transfused_in_48_hours_u__bin_0', 'Blood_transfused_in_48_hours_u__bin_1', 'Platelet_count_x10_3_uL__bin_0', 'Platelet_count_x10_3_uL__bin_1', 'WBC_x10_3_uL__bin_0', 'WBC_x10_3_uL__bin_1', 'Hemoglobin_g_L__bin_0', 'Hemoglobin_g_L__bin_1', 'INR_bin_0', 'INR_bin_1', 'Na_mEq_L__bin_0', 'Na_mEq_L__bin_1', 'Creatinine_mg_L__bin_0', 'Creatinine_mg_L__bin_1', 'Bilirubin_mg_dL__bin_0', 'Bilirubin_mg_dL__bin_1', 'ALT_IU_L__bin_0', 'ALT_IU_L__bin_1', 'Albumin_g_dL__bin_0', 'Albumin_g_dL__bin_1', 'Systolic_blood_pressure_mmHg__bin_0', 'Systolic_blood_pressure_mmHg__bin_1', 'Heart_rate_beats_min__bin_0', 'Heart_rate_beats_min__bin_1', 'Hospitalization_day__bin_0', 'Hospitalization_day__bin_1', 'Etiology_of_cirrhosis_BC', 'Etiology_of_cirrhosis_HBV', 'Etiology_of_cirrhosis_HCV', 'Etiology_of_cirrhosis_NBNC', 'Etiology_of_bleeding

In [33]:
data['X'][0]

array([1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.])

In [34]:
data['Y']

array([[-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.],
      

In [35]:
# --- Problem Parameters ---
max_coefficient = 5          # Maximum absolute value for coefficients
max_L0_value = 20             # Maximum model size (number of non-zero coefficients)
max_offset = 100              # Maximum value for the offset parameter (intercept)
c0_value = 1e-8              # L0-penalty parameter (smaller value gives denser model)

# --- Create Coefficient Set ---
coef_set = riskslim.CoefficientSet(
    variable_names=data['variable_names'],
    lb=np.full(len(data['variable_names']), -max_coefficient, dtype=float),
    ub=np.full(len(data['variable_names']), max_coefficient, dtype=float),
    sign=0  # 0 means no sign restrictions
)
coef_set.update_intercept_bounds(X=data['X'], y=data['Y'], max_offset = max_offset)

# --- Define Constraints ---
constraints = {
    'L0_min': -5,
    'L0_max': max_L0_value,
    'coef_set': coef_set,
}

# --- Define Settings for the Lattice Cutting Plane Algorithm (LCPA) ---
settings = {
    'c0_value': c0_value,
    'max_runtime': 60.0,  # seconds (adjust as needed)
    'max_tolerance': np.finfo('float').eps,
    'display_cplex_progress': True,
    'loss_computation': 'normal',
    'round_flag': True,
    'polish_flag': True,
    'chained_updates_flag': True,
    'add_cuts_at_heuristic_solutions': True,
    'initialization_flag': True,
    'init_max_runtime': 120.0,
    'init_max_coefficient_gap': 0.49,
    'cplex_randomseed': 0,
    'cplex_mipemphasis': 0
}

# settings = {
#     # Problem Parameters
#     'c0_value': c0_value,
#     'w_pos': 1.00,
#     #
#     # LCPA Settings
#     'max_runtime': 300.0,                               # max runtime for LCPA
#     'max_tolerance': np.finfo('float').eps,             # tolerance to stop LCPA (set to 0 to return provably optimal solution)
#     'display_cplex_progress': True,                     # print CPLEX progress on screen
#     'loss_computation': 'normal',                       # how to compute the loss function ('normal','fast','lookup')
#     #
#     # RiskSLIM MIP settings
#     'drop_variables': False,
#     #
#     # LCPA Improvements
#     'round_flag': False,                                # round continuous solutions with SeqRd
#     'polish_flag': False,                               # polish integer feasible solutions with DCD
#     'chained_updates_flag': False,                      # use chained updates
#     'initialization_flag': False,                       # use initialization procedure
#     'init_max_runtime': 300.0,                          # max time to run CPA in initialization procedure
#     'add_cuts_at_heuristic_solutions': True,            # add cuts at integer feasible solutions found using polishing/rounding
#     #
#     # CPLEX Solver Parameters
#     'cplex_randomseed': 0,                              # random seed
#     'cplex_mipemphasis': 0,                             # cplex MIP strategy
# }

# --- Train the RiskSLIM Model using LCPA ---
model_info, mip_info, lcpa_info = riskslim.run_lattice_cpa(data, constraints, settings)

print(model_info['solution'])

# --- Print and Save the Model ---
riskslim.print_model(model_info['solution'], data)
pprint.pprint(model_info)

# Save the model_info to disk for later prediction
with open("riskslim_model.pkl", "wb") as f:
    pickle.dump(model_info, f)



setting c0_value = 0.0 for (Intercept) to ensure that intercept is not penalized
04/13/25 @ 04:11 PM | switching loss computation from normal to fast
04/13/25 @ 04:11 PM | ------------------------------------------------------------
04/13/25 @ 04:11 PM | runnning initialization procedure
04/13/25 @ 04:11 PM | ------------------------------------------------------------
04/13/25 @ 04:11 PM | CPA produced 2 cuts
04/13/25 @ 04:11 PM | running naive rounding on 11 solutions
04/13/25 @ 04:11 PM | best objective value: 0.4509
04/13/25 @ 04:11 PM | rounding produced 5 integer solutions
04/13/25 @ 04:11 PM | best objective value is 0.4498
04/13/25 @ 04:11 PM | running sequential rounding on 11 solutions
04/13/25 @ 04:11 PM | best objective value: 0.4509
04/13/25 @ 04:11 PM | sequential rounding produced 6 integer solutions
04/13/25 @ 04:11 PM | best objective value: 0.2562
04/13/25 @ 04:11 PM | polishing 11 solutions
04/13/25 @ 04:11 PM | best objective value: 0.2562
04/13/25 @ 04:11 PM | poli

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
Tried aggregator 1 time.
Reduced MIP has 92 rows, 94 columns, and 273 nonzeros.
Reduced MIP has 45 binaries, 47 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.15 ticks)
Probing time = 0.00 sec. (0.04 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.12 ticks)
04/13/25 @ 04:11 PM | adding 664 initial cuts

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

      0     0        0.1442    57                      0.1442 

In [36]:
riskslim.print_model(model_info['solution'], data)

+---------------------------------------------+------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-27 + score)) |            |           |
| ==========                                  | ========== | ========= |
| Blood_transfused_in_48_hours_u__bin_0       |   5 points |   + ..... |
| Blood_transfused_in_48_hours_u__bin_1       |   5 points |   + ..... |
| Hemoglobin_g_L__bin_0                       |   5 points |   + ..... |
| Hemoglobin_g_L__bin_1                       |   5 points |   + ..... |
| Bilirubin_mg_dL__bin_1                      |   5 points |   + ..... |
| Hospitalization_day__bin_1                  |   5 points |   + ..... |
| Bilirubin_mg_dL__bin_0                      |   5 points |   + ..... |
| Treatment_APC                               |   3 points |   + ..... |
| ICU_admission                               |   2 points |   + ..... |
| WBC_x10_3_uL__bin_0                         |   1 points |   + ..... |
| Sex                                         |   1

Pr(Y = +1) = 1.0/(1.0 + exp(-(-27 + score)),,
==========,==========,=========
Blood_transfused_in_48_hours_u__bin_0,5 points,+ .....
Blood_transfused_in_48_hours_u__bin_1,5 points,+ .....
Hemoglobin_g_L__bin_0,5 points,+ .....
Hemoglobin_g_L__bin_1,5 points,+ .....
Bilirubin_mg_dL__bin_1,5 points,+ .....
Hospitalization_day__bin_1,5 points,+ .....
Bilirubin_mg_dL__bin_0,5 points,+ .....
Treatment_APC,3 points,+ .....
ICU_admission,2 points,+ .....


In [37]:
solution = model_info['solution']
solution

array([-27.,   1.,   1.,  -0.,   0.,   0.,  -0.,   2.,   0.,   0.,   5.,
         5.,   1.,   0.,   1.,   0.,   5.,   5.,   0.,   0.,   0.,  -1.,
         0.,   1.,   5.,   5.,   0.,   0.,   1.,   0.,   1.,   0.,  -0.,
         0.,   0.,   5.,  -1.,   0.,   1.,   0.,  -2.,   0.,   3.,  -0.,
         0.,   0.])

In [38]:
scores_train = data['X'] @ solution
labels_train = data['Y']


# Round down min and round up max to get threshold range
min_thresh = int(np.floor(np.sum(solution[solution < 0]))) # lowest theoretical score
max_thresh = int(np.ceil(np.sum(solution[solution > 0]))) # highest theoretical score
print(f"Threshold range: {min_thresh} to {max_thresh}")

# Generate integer thresholds in that range
thresholds = range(min_thresh, max_thresh + 1)

# Store results
results = []

for thresh in thresholds:
    preds = np.where(scores_train >= thresh, 1, -1)
    
    tn, fp, fn, tp = confusion_matrix(labels_train, preds, labels=[-1, 1]).ravel()
    
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    youden_index = sensitivity + specificity - 1
    
    results.append({
        "threshold": thresh,
        "youden_index": youden_index,
        "sensitivity": sensitivity,
        "specificity": specificity,
        "precision": tp / (tp + fp) if (tp + fp) > 0 else 0,
        "npv": tn / (tn + fn) if (tn + fn) > 0 else 0,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    })

# Get the threshold with highest Youden index
best_result = max(results, key=lambda x: x["youden_index"])
best_result

Threshold range: -31 to 48


{'threshold': -3,
 'youden_index': np.float64(0.5359940780951207),
 'sensitivity': np.float64(0.8275862068965517),
 'specificity': np.float64(0.7084078711985689),
 'precision': np.float64(0.12834224598930483),
 'npv': np.float64(0.9875311720698254),
 'tp': np.int64(24),
 'tn': np.int64(396),
 'fp': np.int64(163),
 'fn': np.int64(5)}

In [39]:
thresh = best_result['threshold']

preds_train = np.where(scores_train >= thresh, 1, -1)

tn, fp, fn, tp = confusion_matrix(labels_train, preds_train, labels=[-1, 1]).ravel()

accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0

# F1 scores for both classes
f1_pos = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
f1_neg = 2 * (specificity * npv) / (specificity + npv) if (specificity + npv) > 0 else 0
f1_macro = (f1_pos + f1_neg) / 2

# AUC Score
# labels_test should be {1, -1}, convert to {1, 0} for AUC
labels_binary = (labels_train == 1).astype(int)
auc = roc_auc_score(labels_binary, scores_train)

print("=== Train Set Evaluation ===")
print(f"Threshold: {thresh}")
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Sensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Negative Predictive Value: {npv:.3f}")
print(f"F1 Macro Score: {f1_macro:.3f}")
print(f"AUC Score: {auc:.3f}")

=== Train Set Evaluation ===
Threshold: -3
TP: 24, FP: 163, TN: 396, FN: 5
Accuracy: 0.714
Sensitivity (Recall): 0.828
Specificity: 0.708
Precision: 0.128
Negative Predictive Value: 0.988
F1 Macro Score: 0.524
AUC Score: 0.843


In [40]:
# Load test dataset
df_test = pd.read_csv('data_test_infection.csv')
# df_test = pd.read_csv('data_test_rebleeding.csv')
# df_test = pd.read_csv('data_test_mortality.csv')

# Define the target column for the test set
target_col_test = "infection"
# target_col_test = "rebleeding"
# target_col_test = "mortality"

# Identify feature columns (all columns except the target)
feature_cols_test = [col for col in df_test.columns if col != target_col_test]

# Add an intercept column (all ones)
df_test[intercept_col] = 1.0

# Prepend the intercept to the feature list
variable_names_test = [intercept_col] + feature_cols_test

# Create the feature matrix X and target vector Y for the test set
X_test = df_test[variable_names_test].values
y_test = df_test[target_col_test].values

# Create the data dictionary for the test set
data_test = {
    'variable_names': variable_names_test,
    'X': X_test,
    'Y': y_test,
    'outcome_name': target_col_test  
}

print("Variable names (test set):", data_test['variable_names'])
print("X shape (test set):", data_test['X'].shape)
print("Y shape (test set):", data_test['Y'].shape)

data_test['Y']

Variable names (test set): ['(Intercept)', 'Antibiotic_prophylaxis', 'Sex', 'HCC', 'Ascites', 'Hepatic_encephalopathy', 'Prior_SBP', 'ICU_admission', 'Age_bin_0', 'Age_bin_1', 'Blood_transfused_in_48_hours_u__bin_0', 'Blood_transfused_in_48_hours_u__bin_1', 'Platelet_count_x10_3_uL__bin_0', 'Platelet_count_x10_3_uL__bin_1', 'WBC_x10_3_uL__bin_0', 'WBC_x10_3_uL__bin_1', 'Hemoglobin_g_L__bin_0', 'Hemoglobin_g_L__bin_1', 'INR_bin_0', 'INR_bin_1', 'Na_mEq_L__bin_0', 'Na_mEq_L__bin_1', 'Creatinine_mg_L__bin_0', 'Creatinine_mg_L__bin_1', 'Bilirubin_mg_dL__bin_0', 'Bilirubin_mg_dL__bin_1', 'ALT_IU_L__bin_0', 'ALT_IU_L__bin_1', 'Albumin_g_dL__bin_0', 'Albumin_g_dL__bin_1', 'Systolic_blood_pressure_mmHg__bin_0', 'Systolic_blood_pressure_mmHg__bin_1', 'Heart_rate_beats_min__bin_0', 'Heart_rate_beats_min__bin_1', 'Hospitalization_day__bin_0', 'Hospitalization_day__bin_1', 'Etiology_of_cirrhosis_BC', 'Etiology_of_cirrhosis_HBV', 'Etiology_of_cirrhosis_HCV', 'Etiology_of_cirrhosis_NBNC', 'Etiology_

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1

In [41]:
scores_test = data_test['X'] @ solution
labels_test = data_test['Y']
thresh = best_result['threshold']

preds_test = np.where(scores_test >= thresh, 1, -1)

tn, fp, fn, tp = confusion_matrix(labels_test, preds_test, labels=[-1, 1]).ravel()

accuracy = (tp + tn) / (tp + tn + fp + fn)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
npv = tn / (tn + fn) if (tn + fn) > 0 else 0

# F1 scores for both classes
f1_pos = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
f1_neg = 2 * (specificity * npv) / (specificity + npv) if (specificity + npv) > 0 else 0
f1_macro = (f1_pos + f1_neg) / 2

# AUC Score
# labels_test should be {1, -1}, convert to {1, 0} for AUC
labels_binary = (labels_test == 1).astype(int)
auc = roc_auc_score(labels_binary, scores_test)

print("=== Test Set Evaluation ===")
print(f"Threshold: {thresh}")
print(f"TP: {tp}, FP: {fp}, TN: {tn}, FN: {fn}")
print(f"Accuracy: {accuracy:.3f}")
print(f"Sensitivity (Recall): {sensitivity:.3f}")
print(f"Specificity: {specificity:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Negative Predictive Value: {npv:.3f}")
print(f"F1 Macro Score: {f1_macro:.3f}")
print(f"AUC Score: {auc:.3f}")

=== Test Set Evaluation ===
Threshold: -3
TP: 4, FP: 47, TN: 193, FN: 9
Accuracy: 0.779
Sensitivity (Recall): 0.308
Specificity: 0.804
Precision: 0.078
Negative Predictive Value: 0.955
F1 Macro Score: 0.499
AUC Score: 0.669
